R notebook
--

In [ ]:
# Data: electric vehicle charging locations
# Data source: https://openchargemap.org/
# Using Open Charge Map API to download data with R
# https://openchargemap.org/site/develop/api#/operations/get-poi
# Data saved as csv and json formats


In [1]:
install.packages("tidyverse")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
# https://github.com/r-spatial/sf/issues/1572#issuecomment-758858154
system('sudo add-apt-repository ppa:ubuntugis/ubuntugis-unstable')
system('sudo apt-get update')
system('sudo apt-get install libudunits2-dev libgdal-dev libgeos-dev libproj-dev')
install.packages('sf')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘wk’, ‘classInt’, ‘s2’, ‘units’




In [3]:
install.packages("httr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [4]:
install.packages("jsonlite")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [5]:
install.packages("janitor")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘snakecase’




In [6]:
install.packages("tibble")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [7]:
# devtools::install_github("jeroen/jsonlite")

In [9]:
library(tidyverse)
library(sf)
library(httr)
library(jsonlite)
library(janitor)
library(tibble)

In [10]:
id <- "Lewisham"

In [11]:
la <- st_read(paste0("https://ons-inspire.esriuk.com/arcgis/rest/services/Administrative_Boundaries/Local_Authority_Districts_April_2019_Boundaries_UK_BGC/MapServer/0/query?where=UPPER(lad19nm)%20like%20'%25", URLencode(toupper(id), reserved = TRUE), "%25'&outFields=lad19cd,lad19nm,long,lat&outSR=4326&f=geojson"), quiet = TRUE) %>% 
  select(area_code = lad19cd, area_name = lad19nm)

In [14]:
# Submit request to API
request <- GET(url = "https://api.openchargemap.io/v3/poi/?key=123",
               query = list(
                 output = "json",
                 countrycode = "GB",
                 boundingbox = paste0("(", st_bbox(la)[2], ",", st_bbox(la)[1], "),", "(", st_bbox(la)[4], ",", st_bbox(la)[3], ")"),
                 opendata = TRUE,
                 compact = FALSE,
                 verbose = TRUE,
                 comments = FALSE,
                 camelcase = TRUE)
)

In [15]:
# Parse the response and convert to a data frame
response <- content(request, as = "text", encoding = "UTF-8") %>%
  fromJSON(flatten = TRUE) %>%
  as_tibble(.name_repair = make_clean_names) %>%
  select(-id) %>% 
  unnest(connections) 


In [16]:
# Convert to spatial data, clip by boundary and rename variables
sf <- response %>% 
  st_as_sf(crs = 4326, coords = c("address_info_longitude", "address_info_latitude"))  %>% 
  st_intersection(la) %>% 
  mutate_if(is.character, str_trim) %>% 
  mutate(lon = map_dbl(geometry, ~st_coordinates(.x)[[1]]),
         lat = map_dbl(geometry, ~st_coordinates(.x)[[2]])) %>% 
  unite(address, c("address_info_address_line1", "address_info_address_line2", "address_info_town", "address_info_state_or_province"), sep = ", ") %>% 
  mutate(address = str_replace_all(address, ", NA", ""),
         address = str_replace_all(address, ", , ", ", "),
         cost = str_replace_all(usage_cost, "Ã", "")) %>% 
  select(name = address_info_title, 
         points = number_of_points,
         connection_type = connectionType.title,
         kW = powerKW,
         cost,
         address,
         postcode = address_info_postcode,
         operator = operator_info_title,
         website = operator_info_website_url,
         email = operator_info_contact_email,
         updated = date_last_status_update,
         lon, lat) %>%
  arrange(desc(updated))


Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”


In [17]:
# Write results
st_write(sf, "lewisham_electric_vehicle_charging_locations.geojson")
write_csv(st_set_geometry(sf, value = NULL), "lewisham_electric_vehicle_charging_locations.csv")



Writing layer `lewisham_electric_vehicle_charging_locations' to data source 
  `lewisham_electric_vehicle_charging_locations.geojson' using driver `GeoJSON'
Writing 86 features with 13 fields and geometry type Point.
